In [1]:
import sys
sys.path.insert(0, '..')

%load_ext autoreload
%autoreload 2
import vardl
import torch
import torch.nn as nn
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

/home/srossi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### batches = 1
nmc = 10
in_channels = 1
in_height = 6
in_width = 6

input_data = torch.randn([batches, nmc, in_channels, in_height, in_width])

input_data.size()

In [3]:
convlayer = vardl.layers.BayesianConv2d(in_channels=in_channels, in_height=in_height, in_width=in_width, out_channels=1, kernel_size=3, nmc_test=10, nmc_train=10)

In [4]:
convlayer

BayesianConv2d(
  (prior_W): MatrixGaussianDistribution(approx=factorized)
  (q_posterior_W): MatrixGaussianDistribution(approx=factorized)
  (unfold_engine): Unfold(kernel_size=3, dilation=1, padding=1, stride=1)
  (fold_engine): Fold(output_size=(9, 9), kernel_size=1, dilation=1, padding=1, stride=1)
)

In [5]:
convlayer.out_height

6

In [6]:
convlayer(input_data).size()

torch.Size([10, 10, 1, 6, 6])

In [7]:
arch = nn.Sequential(convlayer, 
                    convlayer)
arch

Sequential(
  (0): BayesianConv2d(
    (prior_W): MatrixGaussianDistribution(approx=factorized)
    (q_posterior_W): MatrixGaussianDistribution(approx=factorized)
    (unfold_engine): Unfold(kernel_size=3, dilation=1, padding=1, stride=1)
    (fold_engine): Fold(output_size=(9, 9), kernel_size=1, dilation=1, padding=1, stride=1)
  )
  (1): BayesianConv2d(
    (prior_W): MatrixGaussianDistribution(approx=factorized)
    (q_posterior_W): MatrixGaussianDistribution(approx=factorized)
    (unfold_engine): Unfold(kernel_size=3, dilation=1, padding=1, stride=1)
    (fold_engine): Fold(output_size=(9, 9), kernel_size=1, dilation=1, padding=1, stride=1)
  )
)

In [8]:
model = vardl.models.ClassBayesianNet(arch)

In [9]:
model(input_data).size()

torch.Size([100, 10, 1, 6, 6])

In [10]:
28**2

784

In [1]:
import sys
sys.path.insert(0, '..')

%load_ext autoreload
%autoreload 2
import vardl
import torch
import torch.nn as nn
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

/home/srossi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class View(nn.Module):
    def __init__(self, *size):
        super(View, self, ).__init__()
        self.size = size
   
    def forward(self, x):
        x = x.contiguous().view(self.size) 
        #print('view-output:', x.size())
        return x

In [3]:
X, Y = torch.load('../../datasets/mnist/complete_mnist.pt')
X_test, Y_test = torch.load('../../datasets/mnist/test_mnist.pt')

In [4]:
X = X.view(-1, 1, 28, 28)
X_test = X_test.view(-1, 1, 28, 28)
X.size()

torch.Size([70000, 1, 28, 28])

In [5]:
dataset = TensorDataset(X, Y)

dataloader = DataLoader(dataset, batch_size=64, shuffle=True, 
                              drop_last=False, num_workers=0)

dataloader_test = DataLoader(TensorDataset(X_test, Y_test), 
                        batch_size=64, shuffle=True, 
                              drop_last=False, num_workers=0)

In [6]:
config = {'nmc_train': 1, 'nmc_test': 1, 'local_reparameterization': True}

conv1 = vardl.layers.BayesianConv2d(in_channels=1, 
                                         in_height=28, 
                                         in_width=28,
                                         out_channels=10,
                                         kernel_size=5,
                                         **config)

pool = nn.MaxPool2d(2)


conv2 = vardl.layers.BayesianConv2d(in_channels=10, 
                                         in_height=13, 
                                         in_width=13,
                                         out_channels=20,
                                         kernel_size=5,
                                         **config)


view = View(10, -1, 320)

layer1 = vardl.layers.BayesianLinear(in_features=500, 
                                     out_features=50, 
                                     **config)

layer2 = vardl.layers.BayesianLinear(in_features=50, 
                                       out_features=10, 
                                    **config)



arch = nn.Sequential(
    conv1, 
    View(-1, 10, 26, 26),
    pool,
    nn.ReLU(),
    conv2, 
    View(-1, 20, 11, 11),
    pool, 
    nn.ReLU(),
    View(1, -1, 500), 
    layer1, 
    nn.ReLU(), 
    layer2    
)

arch

Sequential(
  (0): BayesianConv2d(
    (prior_W): MatrixGaussianDistribution(approx=factorized)
    (q_posterior_W): MatrixGaussianDistribution(approx=factorized)
    (unfold_engine): Unfold(kernel_size=5, dilation=1, padding=1, stride=1)
  )
  (1): View()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): ReLU()
  (4): BayesianConv2d(
    (prior_W): MatrixGaussianDistribution(approx=factorized)
    (q_posterior_W): MatrixGaussianDistribution(approx=factorized)
    (unfold_engine): Unfold(kernel_size=5, dilation=1, padding=1, stride=1)
  )
  (5): View()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): ReLU()
  (8): View()
  (9): BayesianLinear(
    in_features=500, out_features=50, bias=False, local_repr=True
    (prior_W): MatrixGaussianDistribution(approx=factorized)
    (q_posterior_W): MatrixGaussianDistribution(approx=factorized)
  )
  (10): ReLU()
  (11): BayesianLinear(
    in_features=50, out_features

In [7]:
model = vardl.models.ClassBayesianNet(arch)

In [8]:
tb_logger = vardl.logger.TensorboardLogger('./work')
trainer = vardl.trainer.TrainerClassifier(model=model, 
                                         train_dataloader=dataloader, 
                                         test_dataloader=dataloader_test, 
                                         optimizer='Adam', 
                                         optimizer_config={'lr':0.1}, 
                                         device='cpu', 
                                         logger=tb_logger,
                                         seed=0)
trainer_logging_config = {'train_verbose': True, 'train_log_interval':100}

In [9]:
torch.nn.functional.max_pool2d(torch.randn(64, 20, 11, 11), 2).size()

torch.Size([64, 20, 5, 5])

In [10]:
initializer = vardl.initializer.XavierNormalInitializer(model=model)#, train_dataloader=dataloader, device='cpu')# tollerance=0.1, max_iter=10, device='cpu')
initializer.initialize()

INFO - Initialization with Xavier-Normal
INFO - Initialization of layer 0
INFO - Initialization of layer 4
INFO - Initialization of layer 9
INFO - Initialization of layer 11
INFO - Initialization done in 0.0016 sec.


In [11]:
trainer.fit(iterations=1000, test_interval=100, train_verbose=False, train_log_interval=100)

Test  || iter=    0   mnll=     6.194   error=   0.899
Test  || iter=  100   mnll=     2.305   error=   0.902
Test  || iter=  200   mnll=     2.306   error=   0.906
Test  || iter=  300   mnll=     2.304   error=   0.900
Test  || iter=  400   mnll=     2.306   error=   0.896
Test  || iter=  500   mnll=     2.304   error=   0.898
Test  || iter=  600   mnll=     2.307   error=   0.898
Test  || iter=  700   mnll=     2.303   error=   0.895
Test  || iter=  800   mnll=     2.303   error=   0.896


KeyboardInterrupt: 

In [32]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

61500

In [ ]:
moe